In [1]:
import pandas as pd
from glob import glob
import numpy as np
pd.set_option('display.max_columns', None)

In [49]:
vegas = pd.read_csv(glob("spre*")[0])
vegas

,schedule_date,schedule_season,schedule_week,schedule_playoff,team_home,score_home,score_away,team_away,team_favorite_id,spread_favorite,over_under_line,stadium,stadium_neutral,weather_temperature,weather_wind_mph,weather_humidity,weather_detail
0,9/2/1966,1966,1,False,Miami Dolphins,14.0,23.0,Oakland Raiders,NaN,NaN,NaN,Orange Bowl,False,83.0,6.0,71,NaN
1,9/3/1966,1966,1,False,Houston Oilers,45.0,7.0,Denver Broncos,NaN,NaN,NaN,Rice Stadium,False,81.0,7.0,70,NaN
2,9/4/1966,1966,1,False,San Diego Chargers,27.0,7.0,Buffalo Bills,NaN,NaN,NaN,Balboa Stadium,False,70.0,7.0,82,NaN
3,9/9/1966,1966,2,False,Miami Dolphins,14.0,19.0,New York Jets,NaN,NaN,NaN,Orange Bowl,False,82.0,11.0,78,NaN
4,9/10/1966,1966,1,False,Green Bay Packers,24.0,3.0,Baltimore Colts,NaN,NaN,NaN,Lambeau Field,False,64.0,8.0,62,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12929,1/3/2021,2020,17,False,New England Patriots,NaN,NaN,New York Jets,NaN,NaN,NaN,Gillette Stadium,False,NaN,NaN,NaN,NaN
12930,1/3/2021,2020,17,False,New York Giants,NaN,NaN,Dallas Cowboys,NaN,NaN,NaN,MetLife Stadium,False,NaN,NaN,NaN,NaN
12931,1/3/2021,2020,17,False,Philadelphia Eagles,NaN,NaN,Washington Football Team,NaN,NaN,NaN,Lincoln Financial Field,False,NaN,NaN,NaN,NaN
12932,1/3/2021,2020,17,False,San Francisco 49ers,NaN,NaN,Seattle Seahawks,NaN,NaN,NaN,Levi's Stadium,False,NaN,NaN,NaN,NaN


In [50]:
#selection of games we are evaluating

reals = vegas[vegas.spread_favorite.notnull()]
data = vegas[vegas["schedule_season"] >= 2002]
data = data[data.score_home.notnull()]
data = data[data["schedule_season"] < 2020]
# reg = data[len(data["schedule_week"]) < 3]
weeks = list(range(1,18))
weeks = list(map(str, weeks))
reg = data[data["schedule_week"].isin(weeks)]

In [51]:
teams = pd.read_csv(glob("nf*")[0])
team_dict = dict(zip(list(teams.team_name), list(teams.team_id)))
reg = reg.replace(team_dict)
small = reg[["schedule_season", "schedule_week", "team_home", "team_away",
             "score_home", "score_away", "team_favorite_id", "spread_favorite"]]

,schedule_season,schedule_week,team_home,team_away,score_home,score_away,team_favorite_id,spread_favorite
7872,2002,1,NYG,SF,13.0,16.0,SF,-4.0
7873,2002,1,BUF,NYJ,31.0,37.0,NYJ,-3.0
7874,2002,1,CAR,BAL,10.0,7.0,PICK,0.0
7875,2002,1,CHI,MIN,27.0,23.0,CHI,-4.5
7876,2002,1,CIN,LAC,6.0,34.0,CIN,-3.0
...,...,...,...,...,...,...,...,...
12662,2019,17,MIN,CHI,19.0,21.0,CHI,-4.5
12663,2019,17,NE,MIA,24.0,27.0,NE,-17.0
12664,2019,17,NYG,PHI,17.0,34.0,PHI,-3.5
12665,2019,17,SEA,SF,21.0,26.0,SF,-3.5


In [52]:
ptdif = small["score_home"] - small["score_away"]
small.insert(6, "pt_dif", ptdif)

In [53]:
#uses multiplier to adjust spread to be a predicted point differential
#of home_team_pts - away_team_points

mult = list(map(int,small.team_favorite_id == small.team_away))
pred = small.spread_favorite
for index, item in enumerate(mult):
    if (item == 0):
        mult[index] = -1
adj = pred*mult
small.insert(7, "adj_spread", adj)

In [57]:
#formats all -0 as 0
small.adj_spread = small.adj_spread + np.zeros(len(small.adj_spread))

/Users/michaelnercessian/opt/anaconda3/lib/python3.8/site-packages/pandas/core/generic.py:5303: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [60]:
small

,schedule_season,schedule_week,team_home,team_away,score_home,score_away,pt_dif,adj_spread,team_favorite_id,spread_favorite
7872,2002,1,NYG,SF,13.0,16.0,-3.0,-4.0,SF,-4.0
7873,2002,1,BUF,NYJ,31.0,37.0,-6.0,-3.0,NYJ,-3.0
7874,2002,1,CAR,BAL,10.0,7.0,3.0,0.0,PICK,0.0
7875,2002,1,CHI,MIN,27.0,23.0,4.0,4.5,CHI,-4.5
7876,2002,1,CIN,LAC,6.0,34.0,-28.0,3.0,CIN,-3.0
...,...,...,...,...,...,...,...,...,...,...
12662,2019,17,MIN,CHI,19.0,21.0,-2.0,-4.5,CHI,-4.5
12663,2019,17,NE,MIA,24.0,27.0,-3.0,17.0,NE,-17.0
12664,2019,17,NYG,PHI,17.0,34.0,-17.0,-3.5,PHI,-3.5
12665,2019,17,SEA,SF,21.0,26.0,-5.0,-3.5,SF,-3.5


In [63]:
#MSE
np.mean((small.pt_dif - small.adj_spread)**2)

179.73003472222223

In [64]:
#L1 error
np.mean(abs(small.pt_dif - small.adj_spread))

10.44162326388889

In [69]:
#sanity check

# tiny = small[0:2]
# np.mean((tiny.pt_dif - tiny.adj_spread)**2)

In [66]:
#ignores the 34 games where Vegas did not pick a winner
picked = small[small.team_favorite_id != "PICK"]

In [67]:
picked.shape

(4574, 10)

In [68]:
#winner accuracy
pred_sign = np.sign(picked.adj_spread)
pt_sign = np.sign(picked.pt_dif)
np.mean(pt_sign == pred_sign)

0.6641888937472672